# Alocação e otimização de portfólios

## Importando libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## Importando os dados

In [2]:
dataset = pd.read_csv('acoes.csv')
dataset

,Date,GOL,CVC,WEG,MGLU,TOTVS,IBOV
0,2015-01-02,14.99,11.164607,4.988705,2.058074,10.155536,47.259998
1,2015-01-05,14.85,11.017703,5.022718,2.096750,9.843493,46.320000
2,2015-01-06,15.21,10.870803,4.948212,2.069124,9.227921,46.580002
3,2015-01-07,14.55,10.775316,4.891521,2.138187,9.162673,48.150002
4,2015-01-08,14.27,10.393366,4.974128,2.121612,9.375432,48.509998
...,...,...,...,...,...,...,...
2477,2025-01-27,1.71,1.860000,53.310001,7.010000,31.209999,121.870003
2478,2025-01-28,1.71,1.770000,54.090000,6.760000,30.790001,121.029999
2479,2025-01-29,1.69,1.800000,54.430000,6.590000,31.150000,120.169998
2480,2025-01-30,1.70,1.950000,56.209999,7.270000,32.610001,123.949997


## Alocação aleatório de ativos

In [23]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0):
    dataset = dataset.copy()

    if seed != 0  :
        np.random.seed(seed)

    pesos = np.random.random(len(dataset.columns) - 1)
    pesos = pesos / pesos.sum()
    print(pesos, pesos.sum())

    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = ( dataset[i] / dataset[i][0])

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['soma valor'] = dataset.iloc[:, 1:].sum(axis = 1)

    datas = dataset['Date']

    dataset.drop(labels= ['Date'], axis = 1, inplace= True)
    dataset['taxa retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i -1]) - 1) * 100

    acoes_pesos = pd.DataFrame(data = {'Ações': colunas, 'Pesos': pesos * 100})

    return dataset, datas, acoes_pesos, dataset.loc[len(dataset) - 1]['soma valor']

In [25]:
dataset, datas, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, 10)

[0.26617196 0.00716121 0.21866313 0.25840174 0.17202779 0.07757418] 1.0


C:\Users\Fabio\AppData\Local\Temp\ipykernel_26308\3372095534.py:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i -1]) - 1) * 100
C:\Users\Fabio\AppData\Local\

In [26]:
dataset

,GOL,CVC,WEG,MGLU,TOTVS,IBOV,soma valor,taxa retorno
0,1330.859777,35.806036,1093.315674,1292.008683,860.138954,387.870876,5000.000000,0.000000
1,1318.430187,35.334900,1100.769962,1316.288700,833.709969,380.156147,4984.689864,-0.306203
2,1350.392100,34.863776,1084.441402,1298.945745,781.573139,382.290029,4932.506190,-1.046879
3,1291.795217,34.557541,1072.016961,1342.301935,776.046897,395.175284,4911.893835,-0.417888
4,1266.935952,33.332587,1090.121024,1331.896641,794.066854,398.129839,4914.482897,0.052710
...,...,...,...,...,...,...,...,...
2477,151.819233,5.965210,11683.325391,4400.707760,2643.379608,1000.207922,19885.405124,-2.296302
2478,151.819233,5.676571,11854.268541,4243.763836,2607.807207,993.313866,19856.649254,-0.144608
2479,150.043571,5.772784,11928.782371,4137.041921,2638.297859,986.255694,19846.194201,-0.052653
2480,150.931402,6.253849,12318.883748,4563.929285,2761.954923,1017.278790,20819.231998,4.902894


In [27]:
datas

0       2015-01-02
1       2015-01-05
2       2015-01-06
3       2015-01-07
4       2015-01-08
           ...    
2477    2025-01-27
2478    2025-01-28
2479    2025-01-29
2480    2025-01-30
2481    2025-01-31
Name: Date, Length: 2482, dtype: object

In [28]:
acoes_pesos

,Ações,Pesos
0,GOL,26.617196
1,CVC,0.716121
2,WEG,21.866313
3,MGLU,25.840174
4,TOTVS,17.202779
5,IBOV,7.757418


In [29]:
soma_valor

np.float64(20797.17212451749)

## Visualização de dados

In [30]:
figura = px.line(x = datas, y = dataset['taxa retorno'], title = 'Retorno diário do Portfólio')
figura.show()

In [33]:
figura = px.line(title = 'Evolução do patrimônio')
for i in dataset.drop(columns= ['soma valor', 'taxa retorno']).columns:
    figura.add_scatter(x = datas, y = dataset[i], name = i)

figura.show()

In [ ]:
figura = px.line(x = datas, y = dataset['soma valor'], title = 'Evoluçào de Patrimônio')
figura.show()